# NASA Firemap
## Install `conda` packages necessary for analysis.

In [ ]:
# Install a conda package in the current Jupyter kernel
import sys

In [ ]:
!conda install --yes --prefix {sys.prefix} numpy

In [ ]:
!conda install --yes --prefix {sys.prefix} pandas

In [ ]:
!conda install --yes --prefix {sys.prefix} scipy

In [ ]:
!conda install --yes --prefix {sys.prefix} matplotlib

In [ ]:
!conda install --yes --prefix {sys.prefix} scikit-learn

In [ ]:
!conda install --yes --prefix {sys.prefix} networkx

In [ ]:
!conda install --yes --prefix {sys.prefix} bokeh

In [ ]:
!conda install --yes --prefix {sys.prefix} beautifulsoup4

In [ ]:
!conda install --yes --prefix {sys.prefix} statsmodels

In [ ]:
!conda install --yes --prefix {sys.prefix} pyspark

In [ ]:
!conda install --yes --prefix {sys.prefix} geopandas

In [ ]:
!conda install --yes --prefix {sys.prefix} cartopy

# Imagery

In [ ]:
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup
import requests

In [ ]:
def download_file(url, filename):
    file_path = './lance-modis/{}'.format(filename)
    r = requests.get(url, allow_redirects=True)
    open(file_path, 'wb').write(r.content)

### Scan remote directory and download

In [ ]:
file_size_wanted = '4096x2048'
def read_url(url):
    url = url.replace(" ","%20")
    req = Request(url)
    a = urlopen(req).read()
    soup = BeautifulSoup(a, 'html.parser')
    x = (soup.find_all('a'))
    
    for i in x:
        file_name = i.extract().get_text()
        url_new = "{}/{}".format(url,file_name) 
        url_new = url_new.replace(" ","%20")

        # only want images
        if url_new.endswith('.jpg'):
            #split the images on .
            img_split = url_new.split('.')
            file_size = img_split[-2]
            
            if file_size == file_size_wanted:
                    try:
                        download_file(url_new, file_name)
                    except Exception:
                        pass  # or you could use 'continue'

read_url("https://lance-modis.eosdis.nasa.gov/imagery/firemaps")

### Create an animated GIF

In [ ]:
!pip install imageio

In [ ]:
import os,sys
import datetime
import imageio
from pprint import pprint
import time
import datetime
import glob
e=sys.exit
 
def create_gif(filenames, duration):
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    output_file = 'Gif-%s.gif' % datetime.datetime.now().strftime('%Y-%M-%d-%H-%M-%S')
    imageio.mimsave(output_file, images, duration=duration)
    

path = "./lance-modis"
filelist = glob.glob(os.path.join(path, '*.jpg'))
filenames = sorted(x for x in filelist)    
duration = 0.2 

create_gif(filenames, duration)

In [ ]:
!pip install imageio-ffmpeg

In [ ]:
#!/usr/bin/env python
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile('lance-modis.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('./lance-modis', zipf)
zipf.close()